In [ ]:
import os, sys
from dotenv import load_dotenv
from snowflake.snowpark import Session
from snowflake.core.task.dagv1 import DAG, DAGTask, DAGOperation
from snowflake.core import CreateMode, Root

In [ ]:
from pathlib import Path
current_dir = Path(os.getcwd())
parent_dir = current_dir.parent.parent
sys.path.append(str(parent_dir))

In [ ]:
load_dotenv()

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

# Build absolute path to key file
key_path = os.environ.get("SNOWFLAKE_PRIVATE_KEY_PATH")

# Handle encrypted or unencrypted private keys
passphrase = os.environ.get("SNOWFLAKE_PRIVATE_KEY_PASSPHRASE")
password = passphrase.encode() if passphrase else None

# Load private key
with open(key_path, "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=password,
        backend=default_backend()
    )

# Convert to bytes format Snowflake expects
pkb = private_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

# Create connection parameters
connection_parameters = {
    "account": os.environ.get("SNOWFLAKE_ACCOUNT"),
    "user": os.environ.get("SNOWFLAKE_USER"),
    "role": os.environ.get("SNOWFLAKE_ROLE"),
    "database": os.environ.get("SNOWFLAKE_DATABASE"),
    "schema": os.environ.get("SNOWFLAKE_SCHEMA"),
    "warehouse": os.environ.get("SNOWFLAKE_WAREHOUSE"),
    "private_key": pkb
}

# Create session
session = Session.builder.configs(connection_parameters).create()
print(f"Connected to: {session.get_current_account()}")

In [ ]:
from src.etl.procs.dag_orchestrator import *

In [ ]:
result = create_dag_orchestrator(session)
print(result)

In [ ]:
# View the DAG
sdf = session.sql("SHOW TASKS IN SCHEMA LEARNING_DB.ETL")
sdf.to_pandas()

In [ ]:
# View the DAG
session.sql("""
SELECT * FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY(TASK_NAME => 'etl_dag_orchestrator')) 
ORDER BY SCHEDULED_TIME DESC
""").to_pandas()


In [ ]:
print(execute_dag(session))

In [ ]:
# View the DAG
session.sql("""
SELECT * FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY(TASK_NAME => 'etl_dag_orchestrator')) 
ORDER BY SCHEDULED_TIME DESC
""").to_pandas()


In [ ]:
# View Dag Operations
session.sql("""
-- Simple query to view recent table_updater procedure queries
SELECT 
    *
FROM TABLE(INFORMATION_SCHEMA.QUERY_HISTORY(
    END_TIME_RANGE_START => DATEADD('hours', -24, CURRENT_TIMESTAMP()),
    RESULT_LIMIT => 100
))
ORDER BY START_TIME DESC;
""").to_pandas()